##Gerar dados fictícios para povoar o dataset

- Uilizar as biblioecas Faker e Random para gerar dados aleatórios, fictícios e com erros propositais para fins educacionais.

In [ ]:
pip install Faker

In [ ]:
import csv, random, json
from faker import Faker
from datetime import datetime, timedelta

In [ ]:
def gerar_nome():
    """Gera nome com possíveis erros simulados."""
    nome = faker.name()
    erros = [
        lambda n: ""                          # nome vazio (erro forçado)
    ]
    if random.random() < random.uniform(0.01, 0.1):
        return random.choice(erros)(nome)
    return nome

def gerar_email():
    """Gera email com possíveis erros simulados."""
    email = faker.email()
    erros = [
        lambda e: e.replace("@", ""),                 # falta @
        lambda e: e.replace(".", ""),                 # falta ponto
        lambda e: e.upper(),                          # caixa alta
        lambda e: e.replace("com", "con"),            # erro digitação
        lambda e: ""                                  # email vazio (erro forçado)
    ]
    if random.random() < random.uniform(0.1, 0.25):
        return random.choice(erros)(email)
    return email

def gerar_idade():
    """Gera idade com possíveis erros simulados."""
    idade = random.randint(18, 70)
    erros = [
        lambda i: -i,          # negativo
        lambda i: i + 100,     # muito alto
        lambda i: str(i),      # como string
        lambda i: "trinta",    # texto
        lambda i: None         # valor nulo (erro forçado)
    ]
    if random.random() < random.uniform(0.1, 0.25):
        return random.choice(erros)(idade)
    return idade

def gerar_cidade(cidades):
    """Gera cidade com possíveis erros simulados."""
    cidade = random.choice(cidades)
    erros = [
        lambda c: c.lower(),           # caixa baixa
        lambda c: c[:5],               # truncada
        lambda c: c.replace("o", "0"), # erro de digitação
        lambda c: ""                   # cidade vazia (erro forçado)
    ]
    if random.random() < random.uniform(0.1, 0.25):
        return random.choice(erros)(cidade)
    return cidade

def gerar_profissao(lista_profissoes):
    """Gera profissão com possíveis erros simulados."""
    profissao = random.choice(lista_profissoes)
    erros = [
        lambda p: p.lower(),   # tudo minúsculo
        lambda p: p.upper(),   # tudo maiúsculo
        lambda p: None         # nulo (erro forçado)
    ]
    if random.random() < random.uniform(0.05, 0.15):
        return random.choice(erros)(profissao)
    return profissao

def gerar_renda():
    """Gera renda com possíveis erros simulados."""
    renda = round(random.uniform(1500, 15000), 2)
    erros = [
        lambda r: -r,                  # negativo
        lambda r: 9999999,             # valor muito alto
        lambda r: "dez mil",           # texto
        lambda r: f"{r:,}",            # separador errado
        lambda r: None                 # nulo (erro forçado)
    ]
    if random.random() < random.uniform(0.1, 0.25):
        return random.choice(erros)(renda)
    return renda

def gerar_clientes(arquivo_csv="001_seed_clientes.csv", num_registros=50):
    """Gera dataset de clientes com inconsistências simuladas."""
    colunas = ["cliente_id", "nome", "email", "idade", "cidade", "profissao", "renda_mensal"]
    cidades = ["São Paulo", "Rio de Janeiro", "Belo Horizonte",
               "Porto Alegre", "Curitiba", "Salvador",
               "Recife", "Brasília", "Manaus",
               "Belém", "Goiânia"]

    lista_profissoes = [faker.job() for _ in range(8)]

    with open(arquivo_csv, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(colunas)

        for i in range(1, num_registros + 1):
            writer.writerow([
                i,
                gerar_nome(),
                gerar_email(),
                gerar_idade(),
                gerar_cidade(cidades),
                gerar_profissao(lista_profissoes),
                gerar_renda()
            ])

    print(f"Arquivo {arquivo_csv} gerado com {num_registros} registros!")

In [ ]:
def gerar_transacoes(arquivo_csv="002_seed_transacoes.csv", num_registros=500):
    """
    Gera um dataset de transações bancárias com possíveis inconsistências.

    - Saques devem ser negativos, mas podem ter erro forçado.
    - Depósitos e transferências devem ser positivos, mas podem ter erro forçado.
    - Datas podem vir em formatos diferentes.
    """
    colunas = ["transacao_id", "cliente_id", "tipo", "valor", "data", "agencia"]
    tipos = ["deposito", "saque", "transferencia"]
    formatos_data = ["%Y-%m-%d", "%d/%m/%Y", "%Y/%m/%d"]

    with open(arquivo_csv, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(colunas)

        for i in range(1, num_registros + 1):
            transacao_id = i
            cliente_id = f"C{str(random.randint(1, 50)).zfill(3)}"
            tipo = random.choice(tipos)

            # Valor correto
            if tipo == "saque":
                valor = -random.randint(100, 2000)
            else:
                valor = random.randint(100, 5000)

            # Forçar erro com probabilidade de 10% a 20%
            if random.random() < random.uniform(0.1, 0.2):
                erros_valor = [
                    lambda v: abs(v),         # sinal invertido (saque positivo, depósito negativo)
                    lambda v: 0,              # valor zerado
                    lambda v: "mil reais",    # valor como texto
                    lambda v: 9999999         # valor absurdamente alto
                ]
                valor = random.choice(erros_valor)(valor)

            # Data aleatória nos últimos 180 dias, com formato inconsistente
            data_base = datetime.today() - timedelta(days=random.randint(0, 180))
            formato_escolhido = random.choice(formatos_data)
            data = data_base.strftime(formato_escolhido)

            # Agência
            agencia = f"AG{str(random.randint(1, 20)).zfill(2)}"

            writer.writerow([transacao_id, cliente_id, tipo, valor, data, agencia])

    print(f"Arquivo {arquivo_csv} gerado com {num_registros} registros!")

In [ ]:
# Uso
faker = Faker('pt_BR')
gerar_clientes("001_seed_clientes.csv", num_registros=50)
gerar_transacoes("002_seed_transacoes.csv", num_registros=500)

Arquivo 001_seed_clientes.csv gerado com 50 registros!
Arquivo 002_seed_transacoes.csv gerado com 500 registros!


# 1. Extração

In [ ]:
import pandas as pd
import re, json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [ ]:
df_clientes = pd.read_csv('001_seed_clientes.csv')
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cliente_id    50 non-null     int64 
 1   nome          46 non-null     object
 2   email         47 non-null     object
 3   idade         49 non-null     object
 4   cidade        45 non-null     object
 5   profissao     49 non-null     object
 6   renda_mensal  47 non-null     object
dtypes: int64(1), object(6)
memory usage: 2.9+ KB


In [ ]:
# Olhando as primeiras linhas do Data Frame
df_clientes.head()

,cliente_id,nome,email,idade,cidade,profissao,renda_mensal
0,1,Maria Flor da Cunha,nalbuquerque@example.org,52,Belo Horizonte,Técnico em hardware,NaN
1,2,Theodoro Vieira,fpastor@exampleorg,42,Salvador,Médico cirurgião,8732.29
2,3,NaN,pguerra@example.net,45,belo horizonte,Taxista,9514.26
3,4,Isabela Pires,josue88@example.com,67,Goiânia,Taxista,12882.33
4,5,Isaque Novaes,dsousa@example.net,34,Recife,NaN,-11720.95


In [ ]:
df_transacoes = pd.read_csv('002_seed_transacoes.csv')
df_transacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   transacao_id  500 non-null    int64 
 1   cliente_id    500 non-null    object
 2   tipo          500 non-null    object
 3   valor         500 non-null    object
 4   data          500 non-null    object
 5   agencia       500 non-null    object
dtypes: int64(1), object(5)
memory usage: 23.6+ KB


In [ ]:
df_transacoes.head()

,transacao_id,cliente_id,tipo,valor,data,agencia
0,1,C042,saque,-1509,16/10/2025,AG15
1,2,C004,saque,0,31/07/2025,AG03
2,3,C003,saque,-189,2025/10/25,AG07
3,4,C001,transferencia,4438,2025/08/08,AG03
4,5,C038,deposito,1571,2025-07-30,AG04


# 2. Transformação

### Validação

In [ ]:
def validar_idade(valor, min_idade=0, max_idade=100):
    """
    Valida idade: deve ser numérica e dentro dos limites.
    """
    try:
        v = int(valor)
        return not (min_idade <= v <= max_idade)
    except:
        return True  # inválido se não for número

def validar_renda(valor, min_renda=0, max_renda=20000):
    """Valida renda: deve ser numérica e dentro dos limites."""
    try:
        v = float(valor)
        return not (min_renda <= v <= max_renda)
    except:
        return True

def validar_email(valor):
    """Valida email: deve conter '@' e '.'."""
    return "@" not in str(valor) or "." not in str(valor)

def relatorio_qualidade(arquivo_csv):
    """
    Gera um relatório de qualidade dos dados para auditoria em ETL.
    """
    df = pd.read_csv(arquivo_csv)

    relatorio = {
        "valores_nulos": df.isnull().sum().to_dict(),
        "duplicados": int(df.duplicated().sum()),
        "id_duplicado": int(df["cliente_id"].duplicated().sum()),
        "cidades_unicas": df["cidade"].value_counts().to_dict(),
        "profissoes_unicas": df["profissao"].value_counts().to_dict(),
        "idades_invalidas": df["idade"].apply(validar_idade).sum(),
        "rendas_invalidas": df["renda_mensal"].apply(validar_renda).sum(),
        "emails_invalidos": df["email"].apply(validar_email).sum()
    }

    return relatorio

In [ ]:
def validar_valor(valor, tipo):
    """Valida valor da transação: saques devem ser negativos, depósitos/transferências positivos."""
    try:
        v = float(valor)
        if tipo == "saque":
            return v >= 0  # erro se saque não for negativo
        else:
            return v <= 0  # erro se depósito/transferência não for positivo
    except:
        return True  # inválido se não for número

def validar_data(valor):
    """Valida se a data pode ser convertida para datetime em algum dos formatos esperados."""
    formatos = ["%d/%m/%Y"]
    for f in formatos:
        try:
            datetime.strptime(str(valor), f)
            return False  # válido
        except:
            continue
    return True  # inválido

def relatorio_qualidade_transacoes(arquivo_csv):
    """
    Gera relatório de qualidade para dataset de transações.
    """
    df = pd.read_csv(arquivo_csv)

    relatorio = {
        "valores_nulos": df.isnull().sum().to_dict(),
        "duplicados": int(df.duplicated().sum()),
        "transacao_id_duplicado": int(df["transacao_id"].duplicated().sum()),
        "tipos_transacao": df["tipo"].value_counts().to_dict(),
        "valores_invalidos": df.apply(lambda row: validar_valor(row["valor"], row["tipo"]), axis=1).sum(),
        "datas_invalidas": df["data"].apply(validar_data).sum(),
        "agencias_unicas": df["agencia"].value_counts().to_dict()
    }

    return relatorio

In [ ]:
# relatório para dados da tabela clientes
arquivo_csv = "001_seed_clientes.csv"
resumo = relatorio_qualidade(arquivo_csv)

print("=== RELATÓRIO DE QUALIDADE DOS DADOS DOS CLIENTES ===")
for chave, valor in resumo.items():
    print(f"{chave}: {valor}")

=== RELATÓRIO DE QUALIDADE DOS DADOS DOS CLIENTES ===
valores_nulos: {'cliente_id': 0, 'nome': 4, 'email': 3, 'idade': 1, 'cidade': 5, 'profissao': 1, 'renda_mensal': 3}
duplicados: 0
id_duplicado: 0
cidades_unicas: {'Goiânia': 7, 'Salvador': 6, 'Belém': 6, 'Rio de Janeiro': 4, 'Brasília': 4, 'Curitiba': 3, 'São Paulo': 3, 'Recife': 3, 'belo horizonte': 2, 'Belo Horizonte': 1, 'curitiba': 1, 'G0iânia': 1, 'manaus': 1, 'porto alegre': 1, 'são paulo': 1, 'Porto': 1}
profissoes_unicas: {'Médico cirurgião': 9, 'Montador de negativos': 9, 'Taxista': 6, 'Lixeiro/Coletor de lixo': 6, 'Comprador': 5, 'Técnico em hardware': 4, 'Epidemiólogo': 3, 'Cartunista': 3, 'TÉCNICO EM HARDWARE': 1, 'montador de negativos': 1, 'MONTADOR DE NEGATIVOS': 1, 'LIXEIRO/COLETOR DE LIXO': 1}
idades_invalidas: 8
rendas_invalidas: 11
emails_invalidos: 7


In [ ]:
# relatório para dados da tabela transações
arquivo_csv = "002_seed_transacoes.csv"
resumo = relatorio_qualidade_transacoes(arquivo_csv)

print("\n=== RELATÓRIO DE QUALIDADE DOS DADOS DAS TRANSAÇÕES ===")
for chave, valor in resumo.items():
    print(f"{chave}: {valor}")


=== RELATÓRIO DE QUALIDADE DOS DADOS DAS TRANSAÇÕES ===
valores_nulos: {'transacao_id': 0, 'cliente_id': 0, 'tipo': 0, 'valor': 0, 'data': 0, 'agencia': 0}
duplicados: 0
transacao_id_duplicado: 0
tipos_transacao: {'transferencia': 177, 'saque': 173, 'deposito': 150}
valores_invalidos: 62
datas_invalidas: 317
agencias_unicas: {'AG03': 37, 'AG09': 35, 'AG13': 32, 'AG10': 32, 'AG15': 31, 'AG04': 31, 'AG07': 30, 'AG17': 29, 'AG12': 27, 'AG08': 24, 'AG16': 24, 'AG05': 21, 'AG20': 21, 'AG02': 20, 'AG11': 19, 'AG19': 19, 'AG01': 18, 'AG18': 17, 'AG14': 17, 'AG06': 16}


### Correção

In [ ]:
def transformar_clientes(arquivo_csv,
                         limites_idade=(0, 100),
                         limites_renda=(0, 20000),
                         cidades_validas=None,
                         preencher_profissao="Não Informado"):

  """
  Transforma o dataset de clientes:
  - Limpa string, converte tipos e aplica regras de idade/renda
  - Normaliza nomes, e-mails, cidades e profissões
  - Valida limites de idade e renda, descarta valores inválidos
  - Opcionalmente restringge cidades a uma lista de válidas
  Retorna DataFrame transformado e resumo de correções.
  """

  # Carrega o CSV
  df = pd.read_csv(arquivo_csv)
  resumo = {}

  # 1) Limpeza e correção de tipos
  # Tipos corretos e correções de nome, e-mail e cidade
  df["nome"] = df["nome"].astype(str).str.strip().str.title() #remover espaço e capitalizar
  df["email"] = df["email"].astype(str).str.strip().str.lower() #remover espaços e minúsculo
  df["cidade"] = df["cidade"].astype(str).str.strip().str.title() # remover espaços e capitalizar
  df["cliente_id"] = pd.to_numeric(df["cliente_id"], errors="coerce").astype(int) #numérico
  resumo['cliente_id_inconvertivel'] = int(df["cliente_id"].isna().sum())

  # 2) Idade: converter para numérico, inválidos viram NaN
  df["idade"] = pd.to_numeric(df["idade"], errors="coerce")
  idades_invalidas = (df["idade"] < limites_idade[0]) | (df["idade"] > limites_idade[1])
  df.loc[idades_invalidas, "idade"] = None
  resumo["idades_invalidas"] = int(idades_invalidas.sum())

  # 3) Cidade válida conforme lista
  if cidades_validas:
    cidades_invalidas = ~df["cidade"].isin(cidades_validas)
    df.loc[cidades_invalidas, "cidade"] = None
    resumo["cidades_invalidas"] = int(cidades_invalidas.sum())

  # 4) Profissão: preencher nulos e outras correções
  df["profissao"] = df["profissao"].astype(str).str.strip()
  profissoes_invalidas = df['profissao'] == ""
  resumo["profissoes_preenchidas"] = int(profissoes_invalidas.sum())
  df.loc[profissoes_invalidas, "profissao"] = preencher_profissao

  # 5) Renda: converter para float, inválidos viram NaN
  df["renda_mensal"] = pd.to_numeric(df["renda_mensal"], errors="coerce")
  rendas_invalidas = (df["renda_mensal"] < limites_renda[0]) | (df["renda_mensal"] > limites_renda[1])
  df.loc[rendas_invalidas, "renda_mensal"] = None
  resumo["rendas_invalidas"] = int(rendas_invalidas.sum())

  # 6) Integridade e consistência
  # - Duplicatas por cliente_id
  id_duplicado = df["cliente_id"].duplicated(keep="first")
  resumo["cliente_id_duplicado"] = int(id_duplicado.sum())
  df = df[~id_duplicado].copy()

  # - Linhas críticas inválidas: id nulo
  linhas_invalidas = df["cliente_id"].isna()
  resumo["linhas_descartadas"] = int(linhas_invalidas.sum())
  df = df[~linhas_invalidas].copy()
  resumo["registros_finais"] = int(len(df))

  return df, resumo

In [ ]:
def transformar_transacoes(arquivo_csv,
                           formatos_data=("%Y-%m-%d", "%d/%m/%Y", "%Y/%m/%d"),
                           tipos_validos=("deposito", "saque", "transferencia"),
                           corrigir_sinal=True):
    """
    Transforma o dataset de transações:
    - Sanitiza strings, converte tipos, aplica regras de valor por tipo.
    - Converte datas para datetime com múltiplos formatos.
    - Deduplica transações e valida IDs.
    Retorna DataFrame transformado e resumo de correções.
    """
    df = pd.read_csv(arquivo_csv)
    resumo = {}

    # 1) Sanitização
    df["cliente_id"] = df["cliente_id"].astype(str).str.strip().str.upper()
    df["tipo"] = df["tipo"].astype(str).str.strip().str.lower()
    df["agencia"] = df["agencia"].astype(str).str.strip().str.upper()

    # 2) Coerção de tipos
    df["transacao_id"] = pd.to_numeric(df["transacao_id"], errors="coerce").astype("Int64")
    df["valor"] = pd.to_numeric(df["valor"], errors="coerce")

    resumo["transacao_id_inconvertivel"] = int(df["transacao_id"].isna().sum())
    resumo["valor_inconvertivel"] = int(df["valor"].isna().sum())

    # 3) Regras de negócio (sinais e limites)
    if corrigir_sinal:
        saque_pos = (df["tipo"] == "saque") & (df["valor"] > 0)
        dep_neg = (df["tipo"].isin(["deposito", "transferencia"])) & (df["valor"] < 0)

        resumo["saques_corrigidos_sinal"] = int(saque_pos.sum())
        resumo["dep_transf_corrigidos_sinal"] = int(dep_neg.sum())

        df.loc[saque_pos, "valor"] = -df.loc[saque_pos, "valor"]
        df.loc[dep_neg, "valor"] = df.loc[dep_neg, "valor"].abs()

    # 4) Conversão de datas (múltiplos formatos)
    def parse_data(x):
        s = str(x).strip()
        for f in formatos_data:
            try:
                return pd.to_datetime(s, format=f)
            except:
                continue
        return pd.NaT

    df["data"] = df["data"].apply(parse_data)
    resumo["datas_invalidas"] = int(df["data"].isna().sum())

    # 5) Normalização de categorias
    # - Tipos inválidos
    tipo_invalido = ~df["tipo"].isin(tipos_validos)
    resumo["tipo_transacao_invalido"] = int(tipo_invalido.sum())
    # Estratégia: manter para revisão ou descartar
    # df = df[~tipo_invalido].copy()

    # 6) Integridade e consistência
    # - Duplicatas por transacao_id
    dup_tx = df["transacao_id"].duplicated(keep="first")
    resumo["transacao_id_duplicado"] = int(dup_tx.sum())
    df = df[~dup_tx].copy()

    # - Linhas críticas inválidas: id nulo, valor NaN, tipo inválido
    linhas_invalidas = df["transacao_id"].isna() | df["valor"].isna() | tipo_invalido.reindex(df.index, fill_value=False)
    resumo["linhas_descartadas"] = int(linhas_invalidas.sum())
    df = df[~linhas_invalidas].copy()

    resumo["registros_finais"] = int(len(df))
    return df, resumo


In [ ]:
# Transformar clientes
df_clientes, resumo_clientes = transformar_clientes(
    "001_seed_clientes.csv",
    limites_idade=(0, 100),
    limites_renda=(0, 20000),
    cidades_validas=["São Paulo","Rio De Janeiro","Belo Horizonte","Porto Alegre",
                     "Curitiba","Salvador","Recife","Brasília","Manaus","Belém","Goiânia"],
    preencher_profissao="Não Informado"
)

# Transformar transações
df_transacoes, resumo_transacoes = transformar_transacoes(
    "002_seed_transacoes.csv",
    formatos_data=("%Y-%m-%d", "%d/%m/%Y", "%Y/%m/%d"),
    tipos_validos=("deposito", "saque", "transferencia"),
    corrigir_sinal=True
)

# Relatórios de transformação
def imprimir_resumo_transformacao(titulo, resumo, salvar_resumo=None):
  """
  Imprime e opcionalmente salva o resumo de transormação

  Parâmetros:
  - título: título do relatório
  - resumo: dicionário com resumo de transformação
  - salvar: caminho do arquivo para salvar o relatório em JSON ou TXT (opcional)

  """
  # Impressão no console de saída
  print("\n" + "="*60)
  print(titulo)
  print("="*60)
  for k, v in resumo.items():
      print(f"- {k}: {v}")

  # Salvamento em arquivo
  if salvar_resumo:
    registro = { "titulo": titulo, "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "resumo": resumo }
    # Se for JSON
    if salvar_resumo.endswith(".json"):
      with open(salvar_resumo, "a", encoding="utf-8") as arquivo:
        arquivo.write(json.dumps(registro, ensure_ascii=False) + "\n")
        # Se for TXT
    elif salvar_resumo.endswith(".txt"):
      with open(salvar_resumo, "a", encoding="utf-8") as arquivo:
        arquivo.write("\n" + "="*60 + "\n")
        arquivo.write(titulo + "\n")
        arquivo.write("="*60 + "\n")
        arquivo.write("Timestamp: " + registro["timestamp"] + "\n")
        for k, v in resumo.items():
          arquivo.write(f"- {k}: {v}\n")
    else:
      raise ValueError("Formato de arquivo não suportado. Use .json ou .txt")

imprimir_resumo_transformacao("RESUMO TRANSFORMAÇÃO CLIENTES", resumo_clientes, salvar_resumo="resumo_clientes.txt")
imprimir_resumo_transformacao("RESUMO TRANSFORMAÇÃO TRANSAÇÕES", resumo_transacoes, salvar_resumo="resumo_transacoes.json")



RESUMO TRANSFORMAÇÃO CLIENTES
- cliente_id_inconvertivel: 0
- idades_invalidas: 5
- cidades_invalidas: 7
- profissoes_preenchidas: 0
- rendas_invalidas: 3
- cliente_id_duplicado: 0
- linhas_descartadas: 0
- registros_finais: 50

RESUMO TRANSFORMAÇÃO TRANSAÇÕES
- transacao_id_inconvertivel: 0
- valor_inconvertivel: 22
- saques_corrigidos_sinal: 13
- dep_transf_corrigidos_sinal: 0
- datas_invalidas: 0
- tipo_transacao_invalido: 0
- transacao_id_duplicado: 0
- linhas_descartadas: 22
- registros_finais: 478


# 3. Carregamento

In [ ]:
def carregar_csv(df, destino="clientes_transformados.csv"):
    """
    Carrega DataFrame transformado em um CSV final.
    """
    df.to_csv(destino, index=False, encoding="utf-8")
    print(f"Dados carregados em {destino} com {len(df)} registros.")


In [ ]:
carregar_csv(df_clientes, "clientes_transformados.csv")

carregar_csv(df_transacoes, "transacoes_transformadas.csv")


Dados carregados em clientes_transformados.csv com 50 registros.
Dados carregados em transacoes_transformadas.csv com 478 registros.


In [ ]:
import sqlite3

def carregar_sql(df, tabela, banco="etl.db"):
    """
    Carrega DataFrame transformado em uma tabela SQL.
    """
    conn = sqlite3.connect(banco)
    df.to_sql(tabela, conn, if_exists="replace", index=False)
    conn.close()
    print(f"Dados carregados na tabela '{tabela}' do banco {banco}.")


In [ ]:
carregar_sql(df_clientes, "clientes")
carregar_sql(df_transacoes, "transacoes")

Dados carregados na tabela 'clientes' do banco etl.db.
Dados carregados na tabela 'transacoes' do banco etl.db.
